In [2]:
import requests
import time
import wget
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException

def get_products_in_page(page):
    list_prod = page.findAll(attrs= {"vocab":"http://schema.org/","class": "row", "typeof": "Product"})
    list_prod = list_prod[0].findAll('div', attrs={'class':'row'})
    prods=[]
    for i in list_prod:
        url_image = i.findAll('img')[0]['src'].replace('ch/C','gr/G')
        title = i.findAll('img')[0]['alt']
        price = float(i.findAll('h2',attrs={"property": "price"})[0].text.strip("$ "))
        prods.append({"title":title,"url_image": url_image,"proce":price,"img_name":url_image.split('/')[-1]})
    return prods

def get_products_with_delay(brower, max_delay):
    for i in range(max_delay):
        page = BeautifulSoup(browser.page_source,"html5lib")
        products = get_products_in_page(page)
        if products:
            break
        time.sleep(1)
    return products
def save_image(path_dir,name)
    url.split('/')[-1]
    url = 'https://farmaciasdesimilares.com/ui/productos/gr/G_119_1.jpg'
    return wget.download(url,out=path_dir)

In [ ]:
catalog_url = 'https://www.farmaciasdesimilares.com/#!/medicamentos'
browser = webdriver.Chrome('./chromedriver')
browser.get(catalog_url)
browser.implicitly_wait(10)

In [ ]:
for i in range(1,10): #82):
    products = get_products_with_delay(brower, 5)
    print("scrappend page: {}".format(i))
    browser.find_element_by_xpath('//*[@id="contenido"]/div/div[4]/div[1]/div/div[1]/dir-pagination-controls/ul/li[9]/a').click()
    time.sleep(1)


In [ ]:
#browser.find_element_by_xpath('//*[@id="contenido"]/div/div[4]/div[1]/div/div[1]/dir-pagination-controls/ul/li[3]/a').click()
time.sleep(2)
browser.find_element_by_xpath('//*[@id="contenido"]/div/div[4]/div[1]/div/div[1]/dir-pagination-controls/ul/li[9]/a').click()

In [ ]:
page = BeautifulSoup(browser.page_source,"html5lib")
products = get_products_in_page(page)

In [ ]:
products

In [ ]:
def get_actions_id_auto_increment(data_list, index, doctype):
    return [{
        "_index": index,
        "_id": _id,
        "_type": doctype,
        "_source": i
    } for i in data_list]

In [ ]:
get_actions_id_auto_increment(products, 'drugstore', 'document')

In [ ]:
def generate_ids(df):
    def increase():
        increase.i += 1
        return increase.i

    increase.i = DataQuality().get_last_id()
    df['id'] = df['id'].apply(lambda _: increase())
    return df

'img//G_119_1.jpg'